In [2]:
# 영화 상영시간 + 잔여 좌석 with pandas

import requests
from bs4 import BeautifulSoup
import pandas as pd

movie = []

url = 'http://www.cgv.co.kr/common/showtimes/iframeTheater.aspx?areacode=01&theatercode=0056&date=20191212'

response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

col_times = soup.find_all('div', {'class':'col-times'})  # 영화당 해당되는 전체 블럭

movie_list = []
movie_time = []
movie_seat = []
movie_dic = {}

movie_max_len = 0    # 영화별 상영 개수가 제일 많은 것의 개수 -> 판다스에서 행의 개수를 하루에 제일 많이 상영하는 영화의 개수에 맞추기 위해서 해줍니다
                     # ex 겨울왕국2 -> 하루 16번 상영 , 나이브스 아웃 8번 상영이면 판다스 표의 행의 개수는 당연히 최소 16개여야겠죠? 

for movie in col_times:
  movie_list.append(movie.find('strong').text.strip())  # 리스트에 영화 제목만 추가


for i in range(len(movie_list)):                        
  movie_time = []
  title = movie_list[i]
  tem = col_times[i].find_all('em')                     # 한 영화에서 em 태그로 감싸진 상영 시간들만 모아서 모두 리스트로 만드는 작업
  



  # 한 영화당 잔여좌석을 리스트로 정리한 작업

  seat = col_times[i].find_all('span')
    
  print(seat)

  new_seat = []

  for index in range(3,len(seat),2):
    each = seat[index].text.strip()
    
    if '잔여좌석' in each:
        new_seat.append(each.split('잔여좌석')[1])
    elif '매진' in each:
        new_seat.append('매진')
   

   # em 태그가 붙어있던 것들을 모두 태그와 쓸데없는 공백은 제거하고 movie_time이라는 리스트에 추가
  print(len(tem), len(new_seat))  

  for i in range(2,len(tem)):
    movie_time.append(tem[i].text.strip() + ' ' + new_seat[i-2])             # 영화 상영시간이랑 해당 시간의 잔여 좌석을 이어주는 작업

  movie_dic[title] = movie_time

  if movie_max_len < len(movie_time):                  
    movie_max_len = len(movie_time)


for key in movie_dic.keys():                # 당일 최대로 상영하는 영화의 개수에 맞춰서 다른 영화들도 상영 시간 개수를 공백(' ')을 이용해서 늘리는 작업
  tem_list = movie_dic[key]
  while True:    
    if len(tem_list) < movie_max_len:
      tem_list.append(' ')      
    else:
      movie_dic[key] = tem_list
      break
  


df = pd.DataFrame(movie_dic, index=[str(i)+'회차' for i in range(1,movie_max_len+1)])

df



[<span class="ico-grade grade-12">
                                        12세 이상</span>, <span class="round lightblue">
<em>
                                            상영중</em>
</span>, <span class="">
<em>
</em>
</span>, <span class="txt-lightblue"><span class="hidden">잔여좌석</span>79석</span>, <span class="hidden">잔여좌석</span>, <span class="txt-lightblue"><span class="hidden">잔여좌석</span>55석</span>, <span class="hidden">잔여좌석</span>, <span class="txt-lightblue"><span class="hidden">잔여좌석</span>126석</span>, <span class="hidden">잔여좌석</span>, <span class="midnight txt-lightblue"><span class="hidden">잔여좌석</span>155석</span>, <span class="hidden">잔여좌석</span>]
6 4
[<span class="ico-grade grade-12">
                                        12세 이상</span>, <span class="round lightblue">
<em>
                                            상영중</em>
</span>, <span class="">
<em>
</em>
</span>, <span class="txt-lightblue"><span class="hidden">잔여좌석</span>57석</span>, <span class="hidden">잔여좌석</span>, <span c

IndexError: list index out of range

In [4]:
# 영화 상영시간 + 잔여 좌석 with pandas (최종판)

import requests
from bs4 import BeautifulSoup
import pandas as pd

movie = []

url = 'http://www.cgv.co.kr/common/showtimes/iframeTheater.aspx?areacode=01&theatercode=0056&date=20191213'

response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

col_times = soup.find_all('div', {'class':'col-times'})  # 영화당 해당되는 전체 블럭

movie_list = []
movie_time = []
movie_seat = []
movie_dic = {}

movie_max_len = 0    # 영화별 상영 개수가 제일 많은 것의 개수 -> 판다스에서 행의 개수를 하루에 제일 많이 상영하는 영화의 개수에 맞추기 위해서 해줍니다
                     # ex 겨울왕국2 -> 하루 16번 상영 , 나이브스 아웃 8번 상영이면 판다스 표의 행의 개수는 당연히 최소 16개여야겠죠? 

for movie in col_times:
  movie_list.append(movie.find('strong').text.strip())  # 리스트에 영화 제목만 추가


for i in range(len(movie_list)):                        
  movie_time = []
  title = movie_list[i]
  tem = col_times[i].select('a > em')                     # 수정 ( a 태그 직계 자식이 em 인 것만 추출 - '준비중' 때문에)
  
    
  # 한 영화당 잔여좌석을 리스트로 정리한 작업

  seat = col_times[i].find_all('span')
  
  
  new_seat = []
  for index in range(3,len(seat)):

    
    each = seat[index].text.strip()
    cnt = 0

    if '잔여좌석' in each:
        a = each.split('잔여좌석')[1]
        if a != '':                                             
            ## ('잔여좌석' 단어만 있는 것이 있어서 split 하면 '' 이 되기 때문에)       -     추가
            new_seat.append(a)   
    elif '매진' in each:
        new_seat.append('매진')
    elif '마감' in each:
        new_seat.append('마감')        
    elif '준비중' in each:
        new_seat.append('준비중')
        
  for i in range(0,len(tem)):
     movie_time.append(tem[i].text.strip() + ' ' + new_seat[i])             # 영화 상영시간이랑 해당 시간의 잔여 좌석을 이어주는 작업

    
    
  movie_dic[title] = movie_time

  if movie_max_len < len(movie_time):                  
    movie_max_len = len(movie_time)

    
    
    

for key in movie_dic.keys():                # 당일 최대로 상영하는 영화의 개수에 맞춰서 다른 영화들도 상영 시간 개수를 공백(' ')을 이용해서 늘리는 작업
  tem_list = movie_dic[key]
  while True:    
    if len(tem_list) < movie_max_len:
      tem_list.append(' ')      
    else:
      movie_dic[key] = tem_list
      break
  


df = pd.DataFrame(movie_dic, index=[str(i)+'회차' for i in range(1,movie_max_len+1)])

df



[<em>16:05</em>, <em>19:10</em>, <em>22:15</em>, <em>25:20</em>]
[<em>16:00</em>, <em>18:40</em>, <em>21:30</em>, <em>24:10</em>, <em>26:50</em>, <em>25:50</em>]
[<em>16:30</em>, <em>18:45</em>, <em>21:00</em>, <em>23:15</em>, <em>25:30</em>, <em>27:45</em>, <em>15:20</em>, <em>17:35</em>, <em>19:50</em>, <em>22:05</em>, <em>24:25</em>, <em>26:40</em>]
[<em>16:25</em>, <em>19:00</em>, <em>21:35</em>, <em>24:20</em>, <em>27:00</em>, <em>23:20</em>, <em>28:30</em>]
[<em>17:10</em>, <em>19:20</em>, <em>21:20</em>]


,포드 V 페라리,나이브스 아웃,겨울왕국 2,쥬만지- 넥스트 레벨,카운트다운
1회차,16:05 79석,16:00 57석,16:30 마감,16:25 84석,17:10 93석
2회차,19:10 55석,18:40 68석,18:45 66석,19:00 73석,19:20 94석
3회차,22:15 126석,21:30 109석,21:00 72석,21:35 101석,21:20 103석
4회차,25:20 155석,24:10 141석,23:15 75석,24:20 124석,
5회차,,26:50 145석,25:30 146석,27:00 124석,
6회차,,25:50 113석,27:45 154석,23:20 마감,
7회차,,,15:20 156석,28:30 108석,
8회차,,,17:35 55석,,
9회차,,,19:50 75석,,
10회차,,,22:05 41석,,


In [72]:
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

wb = Workbook(write_only = True)
ws = wb.create_sheet()


for row in dataframe_to_rows(df, index=True, header=True):
    if len(row) > 1:
        ws.append(row)
        print(row)

wb.save("movie.xlsx")

[None, '나이브스 아웃', '겨울왕국 2', '라스트 크리스마스', '감쪽같은 그녀', '포드 V 페라리', '블랙머니', '러브 앳', '굿 라이어']
['1회차', '07:45 155석', '10:20 133석', '12:20 97석', '07:30 113석', '08:10 121석', '14:35 80석', '07:10 112석', '08:00 144석']
['2회차', '10:25 138석', '12:35 133석', '24:05 113석', '10:00 매진', '17:10 110석', '19:15 75석', '12:40 109석', '26:20 113석']
['3회차', '13:05 123석', '14:50 114석', '28:35 113석', '17:00 ', '20:10 72석', '21:40 103석', '17:30 101석', '15:10 109석']
['4회차', '15:45 89석', '17:15 113석', ' ', ' ', '11:10 100석', '25:20 113석', ' ', ' ']
['5회차', '18:30 99석', '19:30 55석', ' ', ' ', '14:10 86석', ' ', ' ', ' ']
['6회차', '21:10 115석', '21:45 120석', ' ', ' ', '23:10 95석', ' ', ' ', ' ']
['7회차', '23:50 155석', '24:00 141석', ' ', ' ', '26:10 112석', ' ', ' ', ' ']
['8회차', '26:30 156석', '26:15 145석', ' ', ' ', '28:25 145석', ' ', ' ', ' ']
['9회차', '20:00 41석', '07:00 152석', ' ', ' ', '09:40 109석', ' ', ' ', ' ']
['10회차', '22:40 107석', '09:15 142석', ' ', ' ', ' ', ' ', ' ', ' ']
['11회차', '27:45 113석', '11:30 140석', ' ',